# suite2p

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tempfile import TemporaryDirectory
from pathlib import Path
import suite2p
from suite2p import ROI
import plotly.express as px
import plotly.graph_objects as go

ops = suite2p.default_ops()
print(ops)

db = {
    'data_path': [join_file_path('/Users', 'shogoakiyama', 'Desktop')],
    'save_path0': TemporaryDirectory().name,
    'tiff_list': ['Sue_2x_3000_40_-46.tif'],
}

### run 

In [ ]:
output_ops = suite2p.run_s2p(ops=ops, db=db)

In [ ]:
output_op = output_ops
print('param:\n', set(output_op.keys()).difference(ops.keys()))
print('\noutput path: \n', list(Path(output_op['save_path']).iterdir()))

### ROI

In [ ]:


stats_file = Path(output_op['save_path']).joinpath('stat.npy')
iscell = np.load(Path(output_op['save_path']).joinpath('iscell.npy'), allow_pickle=True)[:, 0].astype(bool)
stats = np.load(stats_file, allow_pickle=True)
stats.shape, iscell.shape

In [ ]:
label_id = True
arrays = []
for i, s in enumerate(stats):
    array = ROI(
        ypix=s['ypix'],
        xpix=s['xpix'],
        lam=s['lam'],
        med=s['med'],
        do_crop=False
    ).to_array(Ly=output_op['Ly'], Lx=output_op['Lx'])

    if label_id:
        array *= i + 1

    arrays.append(array)

im = np.stack(arrays)
import copy
nan_im = copy.deepcopy(im)
nan_im[im == 0] = np.nan

### matplot

In [ ]:
plt.imshow(output_op['meanImg'], cmap='gray')
plt.title("Registered Image, Max Projection")

plt.imshow(np.nanmax(nan_im[iscell], axis=0), cmap='jet')
plt.title("All Cell ROIs");

### plotly

In [ ]:
img = output_op['meanImg']
# fig = px.imshow(img, color_continuous_scale='gray')
fig.add_trace(go.Contour(
    z=img,
    showscale=True,
    contours=dict(start=0, end=30, size=30,coloring='lines'),
    line_width=1))
fig.show()

In [ ]:
fig = px.imshow(np.nanmax(nan_im[iscell], axis=0))
fig.show()

# caiman